<a href="https://colab.research.google.com/github/Ethic41/deep_learning/blob/master/pytorch/optimizing_model_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [5]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits



In [7]:
model = NeuralNetwork()

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [9]:
loss_fn = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)

  for batch, (X, y) in enumerate(dataloader):
    # compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")



In [12]:
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}% \
    Avg loss: {test_loss:8f} \n")

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 40

for t in range(epochs):
  print(f"Epoch {t+1}\n--------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
--------------------------------
loss: 0.784533 [    0/60000]
loss: 0.953189 [ 6400/60000]
loss: 0.737664 [12800/60000]
loss: 0.898640 [19200/60000]
loss: 1.070798 [25600/60000]
loss: 0.919807 [32000/60000]
loss: 0.890723 [38400/60000]
loss: 0.985266 [44800/60000]
loss: 0.858531 [51200/60000]
loss: 0.887475 [57600/60000]
Test Error: 
 Accuracy: 70.7%     Avg loss: 0.867338 

Epoch 2
--------------------------------
loss: 0.761236 [    0/60000]
loss: 0.932892 [ 6400/60000]
loss: 0.721218 [12800/60000]
loss: 0.884769 [19200/60000]
loss: 1.056438 [25600/60000]
loss: 0.906004 [32000/60000]
loss: 0.870967 [38400/60000]
loss: 0.979403 [44800/60000]
loss: 0.847249 [51200/60000]
loss: 0.873045 [57600/60000]
Test Error: 
 Accuracy: 71.1%     Avg loss: 0.853105 

Epoch 3
--------------------------------
loss: 0.741205 [    0/60000]
loss: 0.914800 [ 6400/60000]
loss: 0.706793 [12800/60000]
loss: 0.872765 [19200/60000]
loss: 1.043850 [25600/60000]
loss: 0.893822 [32000/60000]
loss: 0.85360